In [1]:
import tensorflow as tf
import numpy as np
import os
import gym

from TNPG import AgentEvaluator, TNPGModel
print('Okay')

/home/yuki/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Okay


In [ ]:
class Args(object):
    iter = 3000
    v_lr = 2e-6
    pi_lr = 1e-6
    gamma = 0.9
    ep_maxlen = 200
    v_iter = 1
    pi_iter = 1
    model_dir = './ckpt/tnpg/'
    logdir = './logs/tnpg/'
    evaluate_every = 50


# Definition of global variables
args = Args()
env_name = 'MountainCarContinuous-v0'
env = gym.make(env_name)
A_DIM = env.action_space.shape[0]
A_LOWER = env.action_space.low[0]
A_HIGHER = env.action_space.high[0]

evaluator = AgentEvaluator()
agent = TNPGModel(args.v_lr, args.pi_lr, args.model_dir)

 [*] Building advantage function...
 [*] Building policy function...
 [*] Building natural gradient...


In [ ]:
s = env.reset()
agent.choose_action(s)

In [ ]:
def collect_one_trajectory(env, agent, maxlen, normalize_state=False, normalize_reward=False):
    """collect_one_trajectory
    :param env: type gym.Env registered
    :param agent: type TNPGModel
    :param maxlen: maximum length of each trajectory
    :param normalize_state: whether normalizing observations
    :param normalize_reward: whether normalizing rewards
    :return: triplet (s, a, r) of type np.array
    """
    s = env.reset()
    # Interact with environment
    buffer_s, buffer_a, buffer_r = [], [], []
    for it in range(maxlen):
        if normalize_state:
            s = np.clip((s - s.mean()) / s.std(), -5.0, 5.0)
        a = agent.choose_action(s)
        buffer_s.append(s[None, :])
        s, r, done, _ = env.step(a)
        buffer_a.append(a[None, :])
        if normalize_reward:
            r = (r - 8.0) / 8.0
        buffer_r.append(r)
        if done:
            break
    # Accumulate rewards
    gamma = 1.0
    for it in range(len(buffer_r) - 2, -1, -1):
        buffer_r[it] = buffer_r[it + 1] + gamma * buffer_r[it]
        gamma *= args.gamma
    states_array = np.concatenate(buffer_s, axis=0)
    actions_array = np.concatenate(buffer_a, axis=0)
    rewards_array = np.array(buffer_r, dtype=np.float32)[:, None]
    return states_array, actions_array, rewards_array


idx = 3
s, a, r = collect_one_trajectory(env, agent, args.ep_maxlen)
feed_dict = {agent.state: s, agent.action: a, agent.reward: r}
g, cg = agent.sess.run([agent.pi_grads[idx], agent.conj_grads[idx][0]], feed_dict=feed_dict)
print('-----------------------------')
print('Variable ' + agent.conj_grads[idx][1].name)
print('Gradient: \n', g)
print('Conjugate gradient: \n', cg)
print('Inner product: ', np.sum(g * cg))

In [5]:
env = env.unwrapped
env.reset()
for it in range(200):
    env.render()
    obs, r, done, info = env.step(env.action_space.sample())
    if done:
        break

NotImplementedError: abstract

In [8]:
s = env.reset()
print(s)
agent.sess.run(agent.value, feed_dict={agent.state: s[None, :]})

[-0.54593634  0.        ]


array([[nan]], dtype=float32)

In [5]:
def hessian_vector_product(x, grad, variable):
    kl_grad_prod = tf.reduce_sum(grad * x)
    return tf.stop_gradient(tf.gradients(kl_grad_prod, variable)[0])


def build_conjugate_gradient(x, kl_grad, variable, n_iter=10, func_Ax=hessian_vector_product):
    """build_conjugate_gradient
    :param x: type tf.Tensor, the initial value of x
    :param kl_grad: type tf.Tensor, the gradient of the objective
    :param variable: type tf.Variable
    :return: the converged conjugate gradient vector \tilde{x} = H^{-1}x

    Truncated natural policy gradient uses fixed number of iterations in the inner loop
    Reference: https://en.wikipedia.org/wiki/Conjugate_gradient_method
    """
    x = tf.stop_gradient(x)
    r = x - func_Ax(x, kl_grad, variable)
    p = tf.stop_gradient(r)
    r_dot_r = tf.reduce_sum(tf.square(r))
    for k in range(n_iter):
        Ap = func_Ax(p, kl_grad, variable)
        p_dot_Ap = tf.reduce_sum(p * Ap)
        alpha = r_dot_r / (p_dot_Ap + 1e-20)
        x = x + alpha * p
        r = r - alpha * Ap
        r_dot_r_new = tf.reduce_sum(tf.square(r))
        beta = r_dot_r_new / (r_dot_rv + 1e-20)
        r_dot_r = r_dot_r_new
        p = r + beta * p
    return x


def conjugate_gradient(x, A, n_iter=30):
    r = x - np.matmul(A, x)
    p = r.copy()
    r_dot_r = np.square(r).sum()
    for k in range(n_iter):
        Ap = np.matmul(A, p)
        p_dot_Ap = np.sum(p * Ap)
        alpha = r_dot_r / p_dot_Ap
        x = x + alpha * p
        r = r - alpha * Ap
        r_dot_r_new = np.square(r).sum()
        beta = r_dot_r_new / r_dot_r
        r_dot_r = r_dot_r_new
        p = r + beta * p
#         print('--------- STEP {} --------'.format(k))
#         print('x: ', x.flatten())
#         print('p: ', p.flatten())
#         print('r: ', r.flatten())
#         print('alpha: ', alpha)
#         print('beta: ', beta)
    return x
    

# params = {'kernel_initializer': tf.orthogonal_initializer()}
# with tf.variable_scope('tested'):
#     x = tf.random_normal([64, 2])
#     h = tf.layers.dense(x, 2, activation=tf.nn.sigmoid, **params)
#     z = tf.layers.dense(h, 1, activation=tf.nn.tanh, **params)
#     y = tf.random_uniform([64], -1.0, 1.0)
#     loss = tf.reduce_mean(tf.square(z - y))


# optim = tf.train.AdamOptimizer(2e-4)
# vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='tested')
# grads_and_vars = optim.compute_gradients(loss, var_list=vars)
# grads = [pair[0] for pair in grads_and_vars]
# conj_grads = []
# for g, var in zip(grads, vars):
#     conj = build_conjugate_gradient(g, g, var)
#     conj_grads.append((conj, var))
# train_op = optim.apply_gradients(conj_grads)
# hessian = tf.hessians(loss, vars)

# sess = tf.Session()
sess.run(tf.global_variables_initializer())

a_vars = sess.run(vars)
H = sess.run(hessian[0])
H = H.reshape((H.shape[0] * H.shape[1], H.shape[2] * H.shape[3]))

print('---------------------------')
print('Variable:\n', a_vars[0])
print('Hessian matrix of rank {}:\n'.format(np.linalg.matrix_rank(H)), H)

g, cg = sess.run([grads[0], conj_grads[0][0]])
cg_array = conjugate_gradient(g.reshape([-1, 1]), H)
newton = np.matmul(np.linalg.inv(H), g.reshape([-1, 1]))

print('Gradient of {}:\n'.format(grads_and_vars[0][1].name), g)
print('Conjugate gradient using TensorFlow:\n', cg)
print('Conjugate gradient using numpy:\n', cg_array)
print('Newton\'s method:\n', newton)

---------------------------
Variable:
 [[-0.04891384 -0.99880296]
 [-0.99880296  0.04891396]]
Hessian matrix of rank 4:
 [[ 0.01015663 -0.01805232  0.00633197  0.00112966]
 [-0.01805232  0.00555454  0.00112966  0.01567117]
 [ 0.00633197  0.00112966 -0.00611497 -0.02305517]
 [ 0.00112966  0.01567117 -0.02305517  0.02079414]]
Gradient of tested/dense/kernel:0:
 [[ 0.04829097 -0.04086177]
 [-0.02593973  0.03705697]]
Conjugate gradient using TensorFlow:
 [[2.6058967  0.77552104]
 [0.8259914  2.8457725 ]]
Conjugate gradient using numpy:
 [[nan]
 [nan]
 [nan]
 [nan]]
Newton's method:
 [[ 4.124737  ]
 [-0.05346113]
 [ 0.47820258]
 [ 2.1284966 ]]


/home/yuki/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in float_scalars
